GRUPO 24:
- Juan David Rodríguez Gómez

- Jonathan David Velosa Lozano

- Anderson Steven Mateus López

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [ ]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction.
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
# ... ENTER YOUR CODE HERE ...
import numpy as np
from numpy.linalg import norm

def jacobi_iteration(A, b, num_iterations):
    n = len(A)
    x = np.zeros(n)
    x_new = np.zeros(n)

    for _ in range(num_iterations):
        for i in range(n):
            x_new[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
        x = x_new.copy()

        B = np.matmul(A, x) - b
        norm_B = norm(B)
        print(f"I {_+1}: Norm[B] = {norm_B}")

    return x

In [ ]:
A = np.array([[3, 1, 2],
              [6, 2, 4],
              [1, 9, -3]])
b = np.array([1, -1, -3])
num_iterations = 10

result = jacobi_iteration(A, b, num_iterations)
print("RESULTADO:", result)

I 1: Norm[B] = 7.457285773732365
I 2: Norm[B] = 29.37397226991236
I 3: Norm[B] = 49.62239444221053
I 4: Norm[B] = 148.78361922022893
I 5: Norm[B] = 391.87568332040627
I 6: Norm[B] = 748.1817010544796
I 7: Norm[B] = 2730.7474587399574
I 8: Norm[B] = 3983.4513468246014
I 9: Norm[B] = 17155.052387360432
I 10: Norm[B] = 27882.341375751916
RESULTADO: [  169.67055327   494.01165981 -4958.91018476]


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration.

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...
def seidel_iteration(A, b, num_iterations):
    n = len(A)
    x = np.zeros(n)

    for _ in range(num_iterations):
        for i in range(n):
            x[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]

        B = np.matmul(A, x) - b
        norm_B = norm(B)
        print(f"I {_+1}: Norm[B] = {norm_B}")

    return x

In [ ]:
n = 5
A = np.random.rand(n, n)
b = np.random.rand(n)
num_iterations = 10

result = seidel_iteration(A, b, num_iterations)
print("RESULTADO: ",result)

I 1: Norm[B] = 32.58234182989987
I 2: Norm[B] = 1370.6919590925513
I 3: Norm[B] = 53132.951553697625
I 4: Norm[B] = 2041447.7524537167
I 5: Norm[B] = 78355349.17621996
I 6: Norm[B] = 3007098199.618231
I 7: Norm[B] = 115403933463.32915
I 8: Norm[B] = 4428869891971.524
I 9: Norm[B] = 169967212692706.03
I 10: Norm[B] = 6522849728716420.0
RESULTADO:  [-1.13457270e+14  3.30481266e+14  3.13041809e+13  5.90986352e+15
 -1.15044859e+16]


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...
from numpy.linalg import norm, solve
def minres(A, b, num_iterations):
    n = len(A)
    x = np.zeros(n)
    r = b - np.dot(A, x)
    p = r.copy()
    norm_b = norm(b)

    for k in range(num_iterations):
        Ap = np.dot(A, p)
        alpha = np.dot(r, r) / np.dot(p, Ap)
        x += alpha * p
        r -= alpha * Ap

        residual_norm = norm(r)
        deviation = norm(x - solve(A, b))
        tau = np.dot(r, Ap) / np.dot(Ap, Ap)

        print(f"I {k+1}: Residual Norm = {residual_norm}, Deviation = {deviation}, Tau = {tau}")

        if residual_norm / norm_b < 1e-6:
            break

        beta = np.dot(r, r) / np.dot(Ap, Ap)
        p = r + beta * p

    return x

In [ ]:
n = 5
A = np.random.rand(n, n)
b = np.random.rand(n)
num_iterations = 10

result = minres(A, b, num_iterations)
print("RESULTADO:", result)

I 1: Residual Norm = 0.6306652198682489, Deviation = 2.9055890934939756, Tau = -0.08148382701126065
I 2: Residual Norm = 2.502721732515086, Deviation = 4.4942536401041595, Tau = -3.1646081659526804
I 3: Residual Norm = 3.2642567476302924, Deviation = 6.252527555876033, Tau = 0.8011787160329858
I 4: Residual Norm = 8.668383457824874, Deviation = 17.56058081355355, Tau = 2.1433156751293163
I 5: Residual Norm = 212.22894807177295, Deviation = 215.51204542992107, Tau = -6.036378911324465
I 6: Residual Norm = 87.59503616062587, Deviation = 39.10893822622091, Tau = -0.0921754382607105
I 7: Residual Norm = 10.64289456867088, Deviation = 16.092476771881753, Tau = -0.011922163915087968
I 8: Residual Norm = 43.06543290497777, Deviation = 40.727026795630834, Tau = 3.925775512929613
I 9: Residual Norm = 22.806399402088857, Deviation = 19.588933279649897, Tau = -0.08234699095740765
I 10: Residual Norm = 14.411912206524113, Deviation = 19.52751068010327, Tau = -0.16302305574256648
RESULTADO: [ 14.64